# ECG Analysis and Visualization

This notebook loads **pre-processed ECG/HRV features** and performs statistical analysis and visualization.

**Prerequisites**: 
- Run `process_ecg_data.py` first to generate processed HRV features
- Install R and required packages for statistical analysis:
  - `lmerTest` (for mixed-effects modeling)
  - `emmeans` (for estimated marginal means and pairwise comparisons)

**Data Source**: Loads from `data/processed/combined/ecg_features_all.csv`

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

print("Libraries imported successfully!")

## 2. Load Pre-Processed HRV Features

Load the combined HRV features file (generated by `process_ecg_data.py`):

In [ ]:
# Load combined HRV features from processed data
features_file = Path('data/processed/combined/ecg_features_all.csv')

if not features_file.exists():
    raise FileNotFoundError(
        f"Processed features file not found: {features_file}\n"
        "Please run 'python process_ecg_data.py' first to generate the features."
    )

all_features_df = pd.read_csv(features_file)

print(f"✓ Loaded {len(all_features_df)} records from {all_features_df['participant'].nunique()} participants")
print(f"✓ Conditions: {sorted(all_features_df['condition'].unique())}")
print(f"✓ Feature columns: {len(all_features_df.columns)}")

## 3. Explore the Data

Display the first few rows and summary statistics:

In [ ]:
# Display first few rows
print("First 10 rows of the dataset:")
display(all_features_df.head(10))

# Display summary statistics for key HRV features
key_features = ['HRV_MeanNN', 'HRV_SDNN', 'HRV_RMSSD', 'HRV_pNN50', 
                'HRV_LF', 'HRV_HF', 'HRV_LFHF', 'HRV_SD1', 'HRV_SD2']
available_features = [f for f in key_features if f in all_features_df.columns]

print("\nSummary statistics for key HRV features:")
display(all_features_df[available_features].describe())

# Display data types and missing values
print("\nData info:")
print(f"Shape: {all_features_df.shape}")
print(f"\nMissing values per column (showing top 10):")
missing_counts = all_features_df.isnull().sum().sort_values(ascending=False)
print(missing_counts.head(10))

## 4. Visualize Key HRV Metrics by Condition

Plot key HRV metrics across workload conditions:

In [ ]:
# Select key HRV metrics to visualize
metrics_to_plot = [
    ('HRV_MeanNN', 'Mean NN Interval (ms)'),
    ('HRV_SDNN', 'SDNN (ms)'),
    ('HRV_RMSSD', 'RMSSD (ms)'),
    ('HRV_pNN50', 'pNN50 (%)'),
    ('HRV_LF', 'LF Power'),
    ('HRV_HF', 'HF Power'),
    ('HRV_LFHF', 'LF/HF Ratio'),
    ('HRV_SD1', 'SD1 (Poincaré)'),
    ('HRV_SD2', 'SD2 (Poincaré)')
]

# Filter to metrics that exist in the data
metrics_to_plot = [(m, t) for m, t in metrics_to_plot if m in all_features_df.columns]

# Create subplots
n_metrics = len(metrics_to_plot)
n_cols = 3
n_rows = (n_metrics + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
axes = axes.flatten() if n_rows > 1 else [axes] if n_metrics == 1 else axes

for idx, (metric, title) in enumerate(metrics_to_plot):
    ax = axes[idx]
    
    # Create box plot for each condition
    data_by_condition = [
        all_features_df[all_features_df['condition'] == cond][metric].dropna()
        for cond in ['L', 'M', 'H']
    ]
    
    bp = ax.boxplot(data_by_condition, labels=['L', 'M', 'H'], patch_artist=True)
    
    # Color the boxes
    colors = ['lightblue', 'lightgreen', 'lightcoral']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
    
    ax.set_xlabel('Workload Condition', fontsize=10)
    ax.set_ylabel(title, fontsize=10)
    ax.set_title(title, fontsize=11, weight='bold')
    ax.grid(axis='y', alpha=0.3)

# Hide any unused subplots
for idx in range(len(metrics_to_plot), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("✓ Visualizations complete")

## 5. Condition-Level Summary Statistics

Compute mean and standard error for each metric by condition:

In [ ]:
# Compute summary statistics by condition for key HRV features
summary_features = {
    'HRV_MeanNN': ['mean', 'std', 'sem'],
    'HRV_SDNN': ['mean', 'std', 'sem'],
    'HRV_RMSSD': ['mean', 'std', 'sem'],
    'HRV_pNN50': ['mean', 'std', 'sem'],
    'HRV_LF': ['mean', 'std', 'sem'],
    'HRV_HF': ['mean', 'std', 'sem'],
    'HRV_LFHF': ['mean', 'std', 'sem'],
    'HRV_SD1': ['mean', 'std', 'sem'],
    'HRV_SD2': ['mean', 'std', 'sem']
}

# Filter to features that exist
summary_features = {k: v for k, v in summary_features.items() if k in all_features_df.columns}

summary_stats = all_features_df.groupby('condition').agg(summary_features).round(3)

print("Summary Statistics by Condition:")
display(summary_stats)

# Also show counts by participant and condition
print("\nRecords per Participant and Condition:")
record_counts = all_features_df.groupby(['participant', 'condition']).size().unstack(fill_value=0)
display(record_counts.describe())

## 6. Add Session Order Information for Statistical Analysis

Load participant info and add session order variables needed for mixed effects models:

In [ ]:
# Load participant info to get session order (using pose utilities)
import sys
sys.path.append('..')
from Pose.utils.io_utils import load_participant_info_file

# Load participant info file
participant_info_path = load_participant_info_file()
participant_info = pd.read_csv(participant_info_path)

print(f"✓ Loaded participant info from: {participant_info_path}")

# Create session_order column (e.g., "LMH", "LHM")
if {"session01", "session02", "session03"}.issubset(participant_info.columns):
    participant_info["session_order"] = (
        participant_info["session01"].str[0] +
        participant_info["session02"].str[0] +
        participant_info["session03"].str[0]
    )
    
    # Map session_order to numeric values
    session_order_numeric_map = {
        "LMH": 1, 
        "LHM": 2, 
        "MLH": 3, 
        "MHL": 4, 
        "HLM": 5, 
        "HML": 6
    }
    
    participant_info["session_order_numeric"] = participant_info["session_order"].map(session_order_numeric_map)
    
    # Create a mapping from participant ID to session_order_numeric
    session_order_map = participant_info.set_index("Participant ID")["session_order_numeric"].to_dict()
    
    # Add session_order_numeric to all_features_df
    all_features_df["session_order_numeric"] = all_features_df["participant"].astype(int).map(session_order_map)
    
    # Add window_index column for compatibility with stats_figures.py
    # ECG features are session-level (not windowed), so set to 0 for all records
    all_features_df["window_index"] = 0
    
    print(f"✓ Added session order information")
    print(f"  Unique session orders: {sorted(participant_info['session_order'].unique())}")
    print(f"  Session order distribution:")
    print(participant_info['session_order'].value_counts().sort_index())
    print(f"✓ Added window_index column (set to 0 for session-level ECG data)")
else:
    raise ValueError("Participant info file missing required columns: session01, session02, session03")

## 7. Statistical Analysis Using Linear Mixed Effects Models

Run mixed effects models and create visualizations for key HRV metrics:

In [ ]:
import sys
sys.path.append('..')  # Add parent directory
from stats_figures import run_rpy2_lmer, barplot_ax

# Define relevant metrics and labels for statistical analysis
metrics = [
    ("HRV_MeanNN", "Mean NN Interval (ms)"),
    ("HRV_SDNN", "SDNN (ms)"),
    ("HRV_RMSSD", "RMSSD (ms)"),
    ("HRV_pNN50", "pNN50 (%)"),
    ("HRV_LF", "LF Power"),
    ("HRV_HF", "HF Power"),
    ("HRV_LFHF", "LF/HF Ratio"),
    ("HRV_SD1", "SD1 (Poincaré)"),
    ("HRV_SD2", "SD2 (Poincaré)")
]

# Filter to metrics that exist in the data
metrics = [(m, l) for m, l in metrics if m in all_features_df.columns]

print(f"✓ Imported statistical utilities")
print(f"  Running analysis on {len(metrics)} HRV metrics")

In [ ]:
# Run statistical analysis for key HRV metrics
for metric, label in metrics:
    print(f"\n{'='*60}")
    print(f"{label}")
    print(f"{'='*60}")
    
    # Run mixed effects model and get stats
    pairwise_p, means, cis = run_rpy2_lmer(
        all_features_df, metric, label
    )
    
    # Prepare data for plotting
    conds = ["L", "M", "H"]
    mean_vals = [means.get(c, float('nan')) for c in conds]
    sems = [(cis[c][1] - cis[c][0]) / 3.92 if c in cis else float('nan') for c in conds]  # 95% CI to SEM
    pvals = [
        pairwise_p.get(("L", "M"), 1.0), 
        pairwise_p.get(("L", "H"), 1.0), 
        pairwise_p.get(("M", "H"), 1.0)
    ]
    
    # Create plot
    fig, ax = plt.subplots(figsize=(5, 6))
    barplot_ax(ax, mean_vals, sems, pvals, ylabel=label, metric_name=metric)
    ax.set_title(label, fontsize=14, weight='bold')
    plt.tight_layout()
    plt.show()

print("\n✓ Statistical analysis complete!")